# Boston Crime Reports db

In this project I built a simple database from from scratch using a dataset from a Boston crime report csv file with the Postgres database engine to perform basic SQL queries. I also created the appropriate datatypes for storing the data and the priviledges for the users (data analysts and data scientists). The advantages of using Postgres as opposed to another database engine such as SQLite is it's flexibility and scalability when it comes to multiple usership. 

I will create 2 user groups:
- readonly: Users in this group can only read the data typically data analysts 
- readwrite: Users in this group have permission to read and change the data.  

## Creating the Database through Python 

I created a database named crime_db and a schema named crimes for storing the tables of the crime data.

As I am yet to make the crime_db I will start by connecting to an existing database called dq.

In [6]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True                           # Autocommit = True is required for creating databases 
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")       
conn.autocommit = False # didn't have to change this but I did as good practice  
conn.close() 

In [7]:
# Connected to the crime database
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;") 
conn.commit()


# Understanding the Data 
 
To understand how to store my data I need to have a point of reference so I can easily remember the type of information that is in the csv file. Therefore, I have written some code below to show the structure of the boston.csv crime data and the amount of unique values that are held under each column. 

In [8]:
# Column headers and 1 row of data
import csv 
with open('boston.csv', 'r') as f:
    reader = csv.reader(f)
    col_headers = next(reader)
    first_row = next(reader) 
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [9]:
# Uniqiue values in each column 
def get_col_value_set(csv_filename, col_index):
    import csv
    values = set()
    with open('boston.csv','r') as f:
        next(f)
        reader =csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values 

for i in range (len(col_headers)):
    values = get_col_value_set('boston.csv', i)
    print(col_headers[i], len(values), sep='\t')
            

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Analysing the Maximum Length of the Description Column

In order to create the appropriate datatypes I need to know specific information from the data in the columns. If I know the maximum length of phrase in the description column I can store it as fixed character length datatype saving memory overall. 

In [10]:
# longest value in 'description' column
crime_descriptions = get_col_value_set('boston.csv', 2)
max_len = 0
for desc in crime_descriptions:
    max_len = max(max_len, len(desc))

print('The maximum length of any value in the description column is', max_len,'characters long.')
    

The maximum length of any value in the description column is 58 characters long.


In [11]:
# print(col_headers)
import csv 
with open('boston.csv', 'r') as f:
    reader = csv.reader(f)
    for i in range(10):
        print(next(reader))

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']
['2', '1402', 'VANDALISM', '2018-08-21', 'Tuesday', '42.30682138', '-71.06030035']
['3', '3410', 'TOWED MOTOR VEHICLE', '2018-09-03', 'Monday', '42.34658879', '-71.07242943']
['4', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.33418175', '-71.07866441']
['5', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.27536542', '-71.09036101']
['6', '3820', 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', '2018-09-03', 'Monday', '42.29019621', '-71.07159012']
['7', '724', 'AUTO THEFT', '2018-09-03', 'Monday', '42.30607218', '-71.0827326']
['8', '3301', 'VERBAL DISPUTE', '2018-09-03', 'Monday', '42.32701648', '-71.10555088']
['9', '301', 'ROBBERY - STREET', '2018-09-03', 'Monday', '42.33152148', '-71.07085307']


## Datatype Choices 

I chose to store 'incident number' as an integer since the numbers look like they are increasing by 1 in each row, there are also around 300k unique values so saving it as a integer will remain within the range. 'Offence code' can be saved as a smallint since the codes seem to range from 3 to 4 digits long. 'description' will be varchar(70) as I know that there is no value in that column which has more than 58 characters. 'date' will be date and 'day_of_the_week' enum as they can only be the 7 days of the week. The 'lat' and 'long' columns will be stored as a decimal where I will set the precision. 

In [12]:
# enum for 7 days of the week 
cur.execute("""
    CREATE TYPE dotw AS ENUM (
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 
    'Friday', 'Saturday', 'Sunday');
""")

cur.execute("""
    CREATE TABLE crimes.boston_crimes(
        incident_number INTEGER PRIMARY KEY,
        offense_code smallint,
        description varchar(70), 
        date date,
        day_of_the_week dotw,
        lat DECIMAL(8,6), 
        long DECIMAL(9,6)
        );
""")

conn.commit()

In [13]:
# check to see in data has been correctly loaded 
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")

print(len(cur.fetchall()))

298329


## Creating Privileges for Groups and Users

To ensure that the users will not inherit any unautherised priveleges on the schema I started by revoking all the privileges from the public group on the public schema.

I also revoked all privileges in the newly created schema. By doing this it makes it easier so I do not need to revoke the privileges when I create users and groups in the future to prevent any privileges from being granted by default.

In [14]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public")

## Creating the read-only and read-write Groups 

As the readonly and read-write groups are not users they don't need login privileges. 

In [27]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# assigning privileges to groups

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Created and Assigned Users - data analyst and data scientist to groups 

In [41]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing the Database setup

I tested the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

For the information_schema.table_privileges I checked the privileges compared to the the SQL queries. 

For the pg_roles table I checked the database related privileges.  

In [46]:
# Check that the privileges have been assigned correctly
cur.execute("""SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly','readwrite');""")
privileges = cur.fetchall()
for row in privileges:
    print(row)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


In [47]:
# Inspecting the privileges related to each group and user 
cur.execute("""
    SELECT rolname, rolsuper, rolinherit, rolcreatedb FROM pg_roles
    WHERE rolname IN ('dq','readonly','readwrite', 'data_analyst', 'data_scientist');
    """)

for user in cur:
    print(user)
print()

('dq', True, True, True)
('readonly', False, True, False)
('readwrite', False, True, False)
('data_analyst', False, True, False)
('data_scientist', False, True, False)



I wanted to see how the other users and groups compared to the super user 'dq'.

## Conclusion 

This project is just a small demonstration of how I can create a database from scratch using any csv file by connecting to the Postgres engine as a server. I installed the PostgreSQL database system and the psycopg2 Python library on my laptop to run it locally on a remote server. This can of course be scaled up when it comes to a multiple user systems. 
